# Improved RAG Pipeline

## Key Features:
1. **Fast API Embeddings**: Support for HuggingFace, Voyage AI, OpenAI, or local FastEmbed
2. **Semantic Chunking**: Respects sentence boundaries with configurable overlap
3. **PyMuPDF**: Faster PDF processing with accurate page number tracking
4. **Cosine Similarity**: Uses IndexFlatIP for better semantic search
5. **Persistence**: Save/load vector store and metadata to avoid re-processing
6. **Progress Tracking**: tqdm progress bars for long operations

## Quick Start:
Set environment variables for API embeddings (optional but recommended):
```bash
# Option 1: HuggingFace (Free with rate limits)
export EMBEDDING_METHOD="huggingface"
export HF_TOKEN="hf_your_token_here"

# Option 2: Voyage AI (Fast, paid)
export EMBEDDING_METHOD="voyage"
export VOYAGE_API_KEY="pa-your-key-here"

# Option 3: OpenAI (Fast, paid)
export EMBEDDING_METHOD="openai"
export OPENAI_API_KEY="sk-your-key-here"

# Option 4: Local FastEmbed (Slow, free, private)
# No setup needed - just don't set any API keys
```

Then run the pipeline cells in order!

In [106]:
# ⚠️ IMPORTANT: Run this cell FIRST to load environment variables from .env
from dotenv import load_dotenv
import os

# Load .env file into the kernel
load_dotenv(override=False)  # Won't overwrite existing environment variables

# Verify key environment variables are loaded (without printing secrets)
print("Environment variables loaded from .env:")
print(f"  ✓ OLLAMA_API_KEY present: {bool(os.environ.get('OLLAMA_API_KEY'))}")
print(f"  ✓ HF_TOKEN present: {bool(os.environ.get('HF_TOKEN'))}")
print(f"  ✓ OPENAI_API_KEY present: {bool(os.environ.get('OPENAI_API_KEY'))}")
print(f"  ✓ VOYAGE_API_KEY present: {bool(os.environ.get('VOYAGE_API_KEY'))}")
print("\nIf any required keys show False, check your .env file and re-run this cell.")

Environment variables loaded from .env:
  ✓ OLLAMA_API_KEY present: True
  ✓ HF_TOKEN present: True
  ✓ OPENAI_API_KEY present: True
  ✓ VOYAGE_API_KEY present: False

If any required keys show False, check your .env file and re-run this cell.


In [107]:
"""
Basic RAG pipeline Implementation:

Documents -> {Chunking / Tokenization -> Convert to Embeddings -> Store Metadata + Chunks in Datastore}
Query -> {Convert to Embedding -> Check for Top K similar documents based on cosine similarity (or other criteria) to query -> Pass Documents to LLM} -> Final Answer with inline citations based on the documents

"""

'\nBasic RAG pipeline Implementation:\n\nDocuments -> {Chunking / Tokenization -> Convert to Embeddings -> Store Metadata + Chunks in Datastore}\nQuery -> {Convert to Embedding -> Check for Top K similar documents based on cosine similarity (or other criteria) to query -> Pass Documents to LLM} -> Final Answer with inline citations based on the documents\n\n'

# Task
Generate a Python template for a RAG pipeline using local Ollama models (gemma3:270m, smollm2:135m, smollm2:360m) for answer generation and a local embedding model (fastembed). Implement the logic for document loading (pdf, md, txt, blobs, docs), chunking (library-based and manual), metadata storage and indexing, embedding generation, vector storage, retrieval, and answer generation. Include necessary library installations and imports, and a simple test.

## Environment setup and model loading

### Subtask:
Install necessary libraries and configure local models via Ollama (gemma3:270m, smollm2:135m/360m). Use fastembed for embeddings to keep everything CPU-only and local.


**Reasoning**:
Install the required libraries for the RAG pipeline.



In [108]:
import os, json
import numpy as np
import faiss
from fastembed import TextEmbedding
from ollama import Client
from tqdm.auto import tqdm
import pickle

# API-based embeddings (faster alternatives)
try:
    import voyageai
    VOYAGE_AVAILABLE = True
except ImportError:
    VOYAGE_AVAILABLE = False

try:
    from openai import OpenAI
    OPENAI_AVAILABLE = True
except ImportError:
    OPENAI_AVAILABLE = False

try:
    from huggingface_hub import InferenceClient
    HF_AVAILABLE = True
except ImportError:
    HF_AVAILABLE = False

# Configuration: Choose embedding method
EMBEDDING_METHOD = os.getenv("EMBEDDING_METHOD", "huggingface")  # Options: "voyage", "openai", "huggingface", "fastembed"
VOYAGE_API_KEY = os.getenv("VOYAGE_API_KEY", None)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
HF_TOKEN = os.getenv("HF_TOKEN", None)

# Local LLMs via Ollama
OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://127.0.0.1:11434")
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "smollm2:360m")
OLLAMA_MODEL_CLOUD = os.getenv("OLLAMA_MODEL_CLOUD", "gpt-oss:20b-cloud")
OLLAMA_API_KEY = os.getenv("OLLAMA_API_KEY", None)  # Required for cloud models
client = Client(host=OLLAMA_BASE_URL)
print(f"Using Ollama model: {OLLAMA_MODEL_CLOUD}")
print(f"OLLAMA_API_KEY configured: {bool(OLLAMA_API_KEY)}")

# Initialize embedding client based on chosen method
if EMBEDDING_METHOD == "voyage" and VOYAGE_AVAILABLE and VOYAGE_API_KEY:
    voyage_client = voyageai.Client(api_key=VOYAGE_API_KEY)
    EMBEDDING_DIM = 1024  # voyage-2 dimension
    print("Using Voyage AI embeddings (1024-dim)")
elif EMBEDDING_METHOD == "openai" and OPENAI_AVAILABLE and OPENAI_API_KEY:
    openai_client = OpenAI(api_key=OPENAI_API_KEY)
    EMBEDDING_DIM = 1536  # text-embedding-3-small dimension
    print("Using OpenAI embeddings (1536-dim)")
elif EMBEDDING_METHOD == "huggingface" and HF_AVAILABLE and HF_TOKEN:
    hf_client = InferenceClient(provider="hf-inference", api_key=HF_TOKEN)
    HF_MODEL = os.getenv("HF_EMBEDDING_MODEL", "BAAI/bge-base-en-v1.5")
    EMBEDDING_DIM = 768  # bge-base dimension
    print(f"Using HuggingFace Inference API with {HF_MODEL} (768-dim)")
else:
    # Fallback to local fastembed
    embedder = TextEmbedding(model_name="BAAI/bge-small-en-v1.5")
    EMBEDDING_DIM = 384
    EMBEDDING_METHOD = "fastembed"
    print("Using local FastEmbed (384-dim) - Set API keys for faster embeddings")
    print("  Options: VOYAGE_API_KEY, OPENAI_API_KEY, or HF_TOKEN")

print("Ollama client ready; embedding system initialized.")


Using Ollama model: gpt-oss:20b-cloud
OLLAMA_API_KEY configured: True
Using HuggingFace Inference API with BAAI/bge-base-en-v1.5 (768-dim)
Ollama client ready; embedding system initialized.


In [109]:
import hashlib
import json
import threading
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path

# ===== EMBEDDING OPTIMIZATION LAYER =====

class EmbeddingCache:
    """
    File-based cache for embeddings using content hash as key.
    Dramatically speeds up repeated queries on same documents.
    """
    def __init__(self, cache_dir: str = ".embedding_cache"):
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)
        self.memory_cache = {}  # In-memory cache for current session
    
    def _hash_text(self, text: str) -> str:
        """Generate hash key from text content."""
        return hashlib.md5(text.encode()).hexdigest()
    
    def get(self, text: str) -> np.ndarray:
        """Retrieve embedding from cache if exists."""
        key = self._hash_text(text)
        
        # Check memory cache first (faster)
        if key in self.memory_cache:
            return self.memory_cache[key]
        
        # Check disk cache
        cache_file = self.cache_dir / f"{key}.npy"
        if cache_file.exists():
            embedding = np.load(cache_file)
            self.memory_cache[key] = embedding  # Cache in memory for this session
            return embedding
        
        return None
    
    def put(self, text: str, embedding: np.ndarray):
        """Store embedding in both memory and disk cache."""
        key = self._hash_text(text)
        self.memory_cache[key] = embedding
        
        # Save to disk
        cache_file = self.cache_dir / f"{key}.npy"
        np.save(cache_file, embedding)

class ParallelHFEmbedder:
    """
    HuggingFace embeddings with parallel processing (4 workers).
    ~5-10x faster than sequential calls for many texts.
    """
    def __init__(self, model: str, api_key: str, num_workers: int = 4):
        self.model = model
        self.api_key = api_key
        self.num_workers = num_workers
        self.client = InferenceClient(provider="hf-inference", api_key=api_key)
    
    def embed_batch(self, texts: list[str]) -> np.ndarray:
        """Embed multiple texts in parallel."""
        results = [None] * len(texts)
        lock = threading.Lock()
        
        def embed_single(idx: int, text: str):
            try:
                embedding = self.client.feature_extraction(text, model=self.model)
                embedding = np.array(embedding, dtype="float32")
                if embedding.ndim > 1:
                    embedding = embedding[0]
                with lock:
                    results[idx] = embedding
            except Exception as e:
                print(f"Error embedding text {idx}: {e}")
                with lock:
                    results[idx] = np.zeros(768, dtype="float32")  # Fallback
        
        # Use thread pool for parallel requests
        with ThreadPoolExecutor(max_workers=self.num_workers) as executor:
            futures = []
            for idx, text in enumerate(texts):
                future = executor.submit(embed_single, idx, text)
                futures.append(future)
            
            # Wait for all to complete
            for future in futures:
                future.result()
        
        return np.array([r for r in results if r is not None], dtype="float32")

class LocalFastEmbedder:
    """
    Local sentence-transformers embeddings with batch processing.
    Faster than API calls but requires local GPU/CPU.
    ~30-60 seconds for 660 chunks on CPU.
    """
    def __init__(self, model_name: str = "BAAI/bge-small-en-v1.5"):
        try:
            from sentence_transformers import SentenceTransformer
            self.model = SentenceTransformer(model_name)
            self.available = True
        except ImportError:
            print("⚠️  sentence-transformers not available. Install with: pip install sentence-transformers")
            self.available = False
    
    def embed_batch(self, texts: list[str], batch_size: int = 32) -> np.ndarray:
        """Embed texts using local sentence-transformers."""
        if not self.available:
            return None
        
        embeddings = self.model.encode(texts, batch_size=batch_size, show_progress_bar=True)
        return np.array(embeddings, dtype="float32")

print("✓ Embedding optimization layer loaded (caching, parallel, local support)")


✓ Embedding optimization layer loaded (caching, parallel, local support)


In [ ]:
    # Embed uncached texts
    if use_parallel and EMBEDDING_METHOD == "huggingface" and HF_AVAILABLE and HF_TOKEN:
        print(f"Embedding {len(uncached_texts)} texts using parallel HuggingFace (4 workers)...")
        parallel_embedder = ParallelHFEmbedder(HF_MODEL, HF_TOKEN, num_workers=4)
        new_embeddings = parallel_embedder.embed_batch(uncached_texts)
    else:
        # Sequential embedding (with all methods supported)
        print(f"Embedding {len(uncached_texts)} texts...")
        new_embeddings = []
        
        for i in tqdm(range(0, len(uncached_texts), batch_size), desc="Embedding", disable=len(uncached_texts) < batch_size):
            batch = uncached_texts[i:i + batch_size]
            
            if EMBEDDING_METHOD == "voyage":
                result = voyage_client.embed(batch, model="voyage-2", input_type="document")
                batch_embeddings = np.array(result.embeddings, dtype="float32")
            elif EMBEDDING_METHOD == "openai":
                result = openai_client.embeddings.create(input=batch, model="text-embedding-3-small")
                batch_embeddings = np.array([e.embedding for e in result.data], dtype="float32")
            elif EMBEDDING_METHOD == "huggingface":
                batch_embeddings = []
                for text in batch:
                    result = hf_client.feature_extraction(text, model=HF_MODEL)
                    embedding = np.array(result, dtype="float32")
                    if embedding.ndim > 1:
                        embedding = embedding[0]
                    batch_embeddings.append(embedding)
                batch_embeddings = np.array(batch_embeddings, dtype="float32")
            else:  # fastembed
                vecs = list(embedder.embed(batch))
                batch_embeddings = np.array([np.asarray(v, dtype="float32") for v in vecs])
            
            new_embeddings.append(batch_embeddings)
        
        new_embeddings = np.vstack(new_embeddings)


✓ Optimized embedding function ready (caching, parallel, local support)


## Document Loading and Preprocessing

### Subtask:
Implement document loading for various file types (pdf, md, txt, blobs, docs) and perform basic preprocessing.

In [111]:
import os
import fitz  # PyMuPDF
import docx
from typing import List, Tuple

def load_document(file_path: str):
    """Loads content from various document types."""
    _, file_extension = os.path.splitext(file_path)
    file_extension = file_extension.lower()

    if file_extension == '.pdf':
        return load_pdf(file_path)
    elif file_extension == '.md':
        return load_text(file_path)
    elif file_extension == '.txt':
        return load_text(file_path)
    elif file_extension == '.docx':
        return load_docx(file_path)
    else:
        raise ValueError(f"Unsupported file type: {file_extension}")

def load_pdf(file_path: str) -> List[Tuple[str, int]]:
    """
    Loads text from a PDF file using PyMuPDF (faster than pypdf).
    Returns list of (text, page_number) tuples to preserve page information.
    """
    try:
        doc = fitz.open(file_path)
        pages_data = []
        
        for page_num in range(len(doc)):
            page = doc[page_num]
            text = page.get_text()
            
            # Clean up text: remove excessive whitespace
            text = ' '.join(text.split())
            
            if text.strip():  # Only add non-empty pages
                pages_data.append((text, page_num + 1))  # 1-indexed page numbers
        
        doc.close()
        print(f"Loaded {len(pages_data)} pages from PDF")
        return pages_data
    except Exception as e:
        print(f"Error loading PDF file {file_path}: {e}")
        return []

def load_text(file_path: str) -> str:
    """Loads text from a plain text file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f"Error loading text file {file_path}: {e}")
        return None

def load_docx(file_path: str) -> str:
    """Loads text from a DOCX file."""
    try:
        doc = docx.Document(file_path)
        text = ""
        for paragraph in doc.paragraphs:
            text += paragraph.text + "\n"
        return text
    except Exception as e:
        print(f"Error loading DOCX file {file_path}: {e}")
        return None

## Chunking Implementation

### Subtask:
Implement document chunking using a library and manually.

In [112]:
import re
from typing import List, Tuple

def chunk_document_semantic(
    document_text: str, 
    chunk_size: int = 1000, 
    overlap: int = 200,
    page_number: int = None
) -> List[Tuple[str, int]]:
    """
    Splits a document into semantic chunks that respect sentence boundaries.
    
    Args:
        document_text: The text to chunk
        chunk_size: Target size for each chunk (in characters)
        overlap: Number of characters to overlap between chunks
        page_number: Page number for this text (if from PDF)
    
    Returns:
        List of (chunk_text, page_number) tuples
    """
    # Split into sentences (handles common sentence endings)
    sentence_endings = re.compile(r'(?<=[.!?])\s+(?=[A-Z])')
    sentences = sentence_endings.split(document_text)
    
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        # If adding this sentence keeps us under chunk_size, add it
        if len(current_chunk) + len(sentence) + 1 <= chunk_size:
            current_chunk += sentence + " "
        else:
            # Save current chunk if it's not empty
            if current_chunk.strip():
                chunks.append((current_chunk.strip(), page_number))
            
            # Start new chunk with overlap from previous chunk
            if overlap > 0 and len(current_chunk) > overlap:
                # Take last 'overlap' characters, but try to start at sentence boundary
                overlap_text = current_chunk[-overlap:]
                # Find the first sentence start in the overlap
                first_sentence_start = overlap_text.find('. ')
                if first_sentence_start != -1:
                    overlap_text = overlap_text[first_sentence_start + 2:]
                current_chunk = overlap_text + sentence + " "
            else:
                current_chunk = sentence + " "
    
    # Add the last chunk
    if current_chunk.strip():
        chunks.append((current_chunk.strip(), page_number))
    
    return chunks

def chunk_pdf_pages(pages_data: List[Tuple[str, int]], chunk_size: int = 1000, overlap: int = 200):
    """
    Chunks PDF pages while preserving page number information.
    
    Args:
        pages_data: List of (text, page_number) tuples from load_pdf
        chunk_size: Target chunk size
        overlap: Overlap between chunks
    
    Returns:
        List of (chunk_text, page_number) tuples
    """
    all_chunks = []
    
    for page_text, page_num in tqdm(pages_data, desc="Chunking pages"):
        page_chunks = chunk_document_semantic(page_text, chunk_size, overlap, page_num)
        all_chunks.extend(page_chunks)
    
    return all_chunks

def chunk_document_manual(document_text: str, max_chars_per_chunk: int = 1000):
    """
    DEPRECATED: Simple character-based chunking (kept for backward compatibility).
    Use chunk_document_semantic() instead for better results.
    """
    print("⚠️  Warning: Using deprecated simple chunking. Consider using chunk_document_semantic()")
    chunks = []
    for i in range(0, len(document_text), max_chars_per_chunk):
        chunks.append(document_text[i:i + max_chars_per_chunk])
    return chunks

## Metadata Storage and Indexing

### Subtask:
Implement a way to store metadata associated with chunks and index them.

In [113]:
import uuid
import pickle
import os

class DocumentChunk:
    def __init__(self, text: str, source: str, page_number: int = None, chunk_id: str = None):
        self.chunk_id = chunk_id if chunk_id is not None else str(uuid.uuid4())
        self.text = text
        self.source = source
        self.page_number = page_number
        self.embedding = None  # To be filled later

class MetadataStore:
    def __init__(self):
        self.chunks = {}  # Stores chunks with chunk_id as key

    def add_chunk(self, chunk: DocumentChunk):
        self.chunks[chunk.chunk_id] = chunk

    def get_chunk(self, chunk_id: str) -> DocumentChunk:
        return self.chunks.get(chunk_id)

    def get_all_chunks(self) -> list[DocumentChunk]:
        return list(self.chunks.values())

    def index_chunks(self):
        """
        Placeholder for more sophisticated indexing.
        Currently chunks are indexed by chunk_id in the dictionary.
        """
        print(f"Indexed {len(self.chunks)} chunks in metadata store.")

    def save(self, save_path: str):
        """Saves metadata store to disk."""
        with open(save_path, 'wb') as f:
            pickle.dump(self.chunks, f)
        print(f"Metadata store saved to {save_path}")

    @classmethod
    def load(cls, save_path: str, verbose: bool = True):
        """Loads metadata store from disk.
        
        Args:
            save_path: Path to metadata file
            verbose: If True, print loading messages. If False, load silently.
        """
        instance = cls()
        with open(save_path, 'rb') as f:
            instance.chunks = pickle.load(f)
        
        if verbose:
            print(f"Metadata store loaded from {save_path} ({len(instance.chunks)} chunks)")
        return instance


## Embedding Generation

### Subtask:
Implement embedding generation for document chunks using the selected embedding model.

In [ ]:
import numpy as np

def generate_embeddings(chunks: list[DocumentChunk], batch_size: int = 100, use_optimizations: bool = True):
    """
    Generates embeddings for chunks using configured method with optimizations.
    
    Optimizations include:
    - Caching: File-based cache for repeated texts (10x+ speedup)
    - Parallelization: Thread pool for HuggingFace API calls (5-10x speedup)
    - Local fast: Sentence-transformers batch processing (fastest, no API)
    
    Args:
        chunks: List of DocumentChunk objects to embed
        batch_size: Batch size for processing
        use_optimizations: Whether to enable caching, parallel, and local embeddings
    """
    texts = [chunk.text for chunk in chunks]
    
    print(f"Generating embeddings for {len(texts)} chunks using optimized {EMBEDDING_METHOD}...")
    print(f"  Options: caching (10x+), parallel HF (5-10x), local fast embedding")
    embeddings = embed_texts_optimized(
        texts, 
        batch_size=batch_size,
        use_cache=use_optimizations,           # Enable caching
        use_parallel=use_optimizations,        # Enable parallelization
        use_local_fast=False                   # Keep False for API methods, True for local
    )
    
    # Assign embeddings to chunks
    for chunk, embedding in zip(chunks, embeddings):
        chunk.embedding = embedding
    
    print(f"✓ Generated {len(embeddings)} embeddings ({EMBEDDING_DIM}-dimensional)")


## Vector Store Implementation

### Subtask:
Implement a vector store for efficient similarity search. Using FAISS for this example.

In [115]:
import faiss
import numpy as np
import pickle
import os

class VectorStore:
    def __init__(self, dimension: int):
        """
        Initialize a Faiss index using Inner Product (cosine similarity after normalization).
        IndexFlatIP is better for embeddings than IndexFlatL2.
        """
        self.dimension = dimension
        self.index = faiss.IndexFlatIP(dimension)  # Inner Product = cosine similarity with normalized vectors
        self.chunk_ids = []  # Maps Faiss index to chunk IDs

    def add_vectors(self, embeddings: np.ndarray, chunk_ids: list[str]):
        """
        Adds embeddings to the Faiss index after L2 normalization.
        Normalization converts inner product to cosine similarity.
        """
        # Normalize embeddings for cosine similarity
        faiss.normalize_L2(embeddings)
        self.index.add(embeddings)
        self.chunk_ids.extend(chunk_ids)
        print(f"Added {len(chunk_ids)} vectors. Total: {self.index.ntotal}")

    def search(self, query_embedding: np.ndarray, k: int = 5):
        """
        Searches for the nearest neighbors using cosine similarity.
        Returns list of chunk IDs sorted by relevance.
        """
        # Normalize query embedding
        query_embedding = query_embedding.reshape(1, -1).astype('float32')
        faiss.normalize_L2(query_embedding)
        
        # Search (higher scores = more similar for IP)
        distances, indices = self.index.search(query_embedding, min(k, self.index.ntotal))

        # Retrieve corresponding chunk IDs
        results = []
        for i in indices[0]:
            if i != -1 and i < len(self.chunk_ids):
                results.append(self.chunk_ids[i])
        return results

    def save(self, save_dir: str):
        """Saves the vector store to disk."""
        os.makedirs(save_dir, exist_ok=True)
        
        # Save FAISS index
        index_path = os.path.join(save_dir, "faiss_index.bin")
        faiss.write_index(self.index, index_path)
        
        # Save chunk IDs and metadata
        metadata_path = os.path.join(save_dir, "chunk_ids.pkl")
        with open(metadata_path, 'wb') as f:
            pickle.dump({
                'chunk_ids': self.chunk_ids,
                'dimension': self.dimension
            }, f)
        
        print(f"Vector store saved to {save_dir}")

    @classmethod
    def load(cls, save_dir: str, verbose: bool = True):
        """Loads the vector store from disk.
        
        Args:
            save_dir: Directory containing saved vector store
            verbose: If True, print loading messages. If False, load silently.
        """
        index_path = os.path.join(save_dir, "faiss_index.bin")
        metadata_path = os.path.join(save_dir, "chunk_ids.pkl")
        
        # Load metadata
        with open(metadata_path, 'rb') as f:
            metadata = pickle.load(f)
        
        # Create instance and load index
        instance = cls(metadata['dimension'])
        instance.index = faiss.read_index(index_path)
        instance.chunk_ids = metadata['chunk_ids']
        
        if verbose:
            print(f"Vector store loaded from {save_dir} ({instance.index.ntotal} vectors)")
        return instance


## Retrieval Implementation

### Subtask:
Implement the retrieval of relevant document chunks based on a query.

In [116]:
def retrieve_documents(query: str, vector_store: VectorStore, metadata_store: MetadataStore, k: int = 5):
    """
    Retrieves the top K most relevant document chunks for a given query using fastembed for the query embedding.
    """
    # 1. Generate embedding for the query (fastembed)
    qv = embed_texts([query])
    # 2. Search the vector store for similar chunks
    retrieved_chunk_ids = vector_store.search(qv[0], k=k)
    # 3. Retrieve the actual chunk objects from the metadata store
    retrieved_chunks = [metadata_store.get_chunk(chunk_id) for chunk_id in retrieved_chunk_ids if metadata_store.get_chunk(chunk_id) is not None]
    return retrieved_chunks

## Answer Generation Implementation

### Subtask:
Implement answer generation using a language model and the retrieved chunks.

In [128]:
from ollama import Client
import os

SYSTEM_PROMPT = (
    "You are a precise assistant. Answer ONLY using the provided sources.\n"
    "Cite evidence with bracketed indices like [1], [2]. If unsure, say you don't know.\n"
    "Keep it concise: 3-6 sentences."
)

# Token limit configuration for LLM responses
# Base tokens for the answer, plus additional tokens per source
TOKEN_LIMIT_BASE = 512          # Minimum tokens for answer generation
TOKEN_LIMIT_PER_SOURCE = 200    # Additional tokens per source (for citations, context)
TOKEN_LIMIT_MAX = 2048          # Absolute maximum to prevent runaway responses

OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://127.0.0.1:11434")
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "smollm2:360m")
OLLAMA_MODEL_CLOUD = os.getenv("OLLAMA_MODEL_CLOUD", "gpt-oss:20b-cloud")
OLLAMA_API_KEY = os.getenv("OLLAMA_API_KEY", None)

try:
    client
except NameError:
    client = Client(host=OLLAMA_BASE_URL)


def generate_answer(query: str, retrieved_chunks: list, model_name: str = None, stream: bool = False):
    """
    Generate an answer from Ollama or Ollama cloud, with defensive checks.

    - Handles None chunk.text
    - Validates cloud API key before using cloud model
    - Uses == for string comparison
    - Supports stream flag if the client and model support streaming
    - Dynamically adjusts token limit based on number of sources
    """
    if model_name is None:
        model_name = os.getenv("OLLAMA_MODEL", "smollm2:360m")

    # Build context lines safely
    context_lines = []
    for i, chunk in enumerate(retrieved_chunks, start=1):
        src = getattr(chunk, "source", f"doc_{i}")
        # ensure chunk.text is a string
        chunk_text_raw = getattr(chunk, "text", "")
        if chunk_text_raw is None:
            chunk_text = ""
        else:
            chunk_text = str(chunk_text_raw)
        # trim to avoid sending too much
        context_lines.append(f"[{i}] {src}:\n{chunk_text[:800]}")

    user = f"Question: {query}\n\nSources:\n" + "\n\n".join(context_lines) + "\n\nAnswer:"

    # Calculate dynamic token limit based on number of sources
    num_sources = len(retrieved_chunks)
    num_predict = min(
        TOKEN_LIMIT_BASE + (num_sources * TOKEN_LIMIT_PER_SOURCE),
        TOKEN_LIMIT_MAX
    )
    
    print(f"[LLM Config] Sources: {num_sources} | Max tokens: {num_predict}")

    # Decide which client to use
    # If using cloud model name, ensure API key exists
    if model_name == OLLAMA_MODEL_CLOUD:
        if not OLLAMA_API_KEY:
            raise ValueError("OLLAMA_API_KEY is not set but cloud model was requested. Set OLLAMA_API_KEY or use the local model.")
        # create a client configured for cloud (keep default client for local)
        cloud_client = Client(host="https://ollama.com", headers={"Authorization": "Bearer " + OLLAMA_API_KEY})
        chosen_client = cloud_client
    else:
        chosen_client = client

    # Prepare messages
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user},
    ]

    # Try streaming if requested and the client supports it
    try:
        if stream:
            # Some clients support streaming via `stream=True` or a stream() method
            # We'll attempt a streaming call but fall back to non-streaming safely
            resp_stream = chosen_client.chat(model=model_name, messages=messages, stream=True, options={"temperature": 0.2, "num_predict": num_predict})
            # If resp_stream is an iterator of chunks/dicts, iterate and concatenate
            full_text = ""
            try:
                for part in resp_stream:
                    # Some streaming responses yield dicts with 'message' -> 'content'
                    if isinstance(part, dict) and "message" in part and isinstance(part["message"], dict):
                        delta = part["message"].get("content", "")
                    elif isinstance(part, str):
                        delta = part
                    else:
                        delta = ""
                    print(delta, end="", flush=True)
                    full_text += delta
                print()
                return full_text.strip()
            except TypeError:
                # Not iterable; fall back
                pass

        # Non-streaming call (or fallback)
        resp = chosen_client.chat(model=model_name, messages=messages, options={"temperature": 0.2, "num_predict": num_predict})
        # resp may be a dict like {'message': {'content': '...'}}
        if isinstance(resp, dict):
            message = resp.get("message")
            if isinstance(message, dict):
                return message.get("content", "").strip()
            # sometimes response might be {'content': '...'}
            return resp.get("content", "").strip()

        # If resp is a string, return it
        if isinstance(resp, str):
            return resp.strip()

        # Unknown shape
        return str(resp)

    except Exception as e:
        # Don't raise raw exceptions to the user; return a helpful message
        return f"Error generating answer: {e}"


In [ ]:
def format_answer_with_sources(answer_text: str, retrieved_chunks: list) -> str:
    """
    Format answer with source citations [1], [2], etc. and source details below.
    
    Example output:
    ---
    The Four Laws of Behavior Change are cue, craving, response, and reward [1]. 
    Each law builds on the previous one to form a complete habit loop [2].
    
    📚 SOURCES:
    [1] Atomic_Habits_James_Clear.pdf (Page 42):
        "The four laws of behavior change are the cue, the craving, the response..."
    
    [2] Atomic_Habits_James_Clear.pdf (Page 45):
        "Together, these four elements form the habit loop which is the..."
    ---
    """
    if not retrieved_chunks or not answer_text.strip():
        return answer_text
    
    # Format source information
    sources_section = "\n📚 SOURCES:\n"
    
    for i, chunk in enumerate(retrieved_chunks, 1):
        source_name = getattr(chunk, "source", f"Document {i}")
        page_num = getattr(chunk, "page_number", None)
        chunk_text = getattr(chunk, "text", "")
        
        # Safely handle None text
        if chunk_text is None:
            chunk_text = "(No text content)"
        else:
            chunk_text = str(chunk_text)
        
        # Truncate text snippet to 200 characters for readability
        snippet = chunk_text[:200].replace("\n", " ")
        if len(chunk_text) > 200:
            snippet += "..."
        
        # Format source entry
        page_info = f" (Page {page_num})" if page_num else ""
        sources_section += f"\n[{i}] {source_name}{page_info}:\n"
        sources_section += f'    "{snippet}"\n'
    
    # Combine answer with sources
    formatted = f"{answer_text}\n{sources_section}"
    return formatted

print("✓ Formatted answer function loaded (with [1], [2] citations and sources)")


✓ Formatted answer function loaded (with [1], [2] citations and sources)


In [129]:
# ===== CENTRALIZED CONFIGURATION =====

import re

# Document & Processing Config
DOCUMENT_CONFIG = {
    "sample_path": "/home/agupta/Documents/Books/Atomic_Habits_James_Clear.pdf",
    "save_dir": "./rag_data",
    "chunk_size": 1000,
    "chunk_overlap": 200,
    "top_k": 5,
}

# Model Config
MODEL_CONFIG = {
    "ollama_local": os.getenv("OLLAMA_MODEL", "smollm2:360m"),
    "ollama_cloud": os.getenv("OLLAMA_MODEL_CLOUD", "gpt-oss:20b-cloud"),
    "api_key": os.getenv("OLLAMA_API_KEY", None),
}

# LLM Response Token Limits
# These control how long the LLM can generate responses
# More sources = more tokens allowed (for proper citations)
# Formula: max_tokens = min(BASE + (num_sources * PER_SOURCE), MAX)
TOKEN_CONFIG = {
    "base": 512,           # Minimum tokens for answer generation (was 256, now 512)
    "per_source": 200,     # Additional tokens per source for citations (~3-5 per citation)
    "max": 2048,           # Absolute maximum to prevent runaway responses
}

# Select model: prefer cloud if API key available, else local
DEFAULT_OLLAMA_MODEL = MODEL_CONFIG["ollama_cloud"] if MODEL_CONFIG["api_key"] else MODEL_CONFIG["ollama_local"]

# ===== REUSABLE HELPER FUNCTIONS =====

def extract_content(resp):
    """
    Extract answer text from various response formats (dict, string, JSON).
    Handles responses from local Ollama, cloud Ollama, and API models.
    """
    if resp is None:
        return ""
    
    # Direct dict-like response
    if isinstance(resp, dict):
        msg = resp.get("message") or resp.get("content")
        if isinstance(msg, dict):
            return msg.get("content", "").strip()
        if isinstance(msg, str):
            return msg.strip()
        return str(resp).strip()
    
    # If it's not a string, stringify it
    if not isinstance(resp, str):
        return str(resp).strip()
    
    s = resp
    
    # Try JSON parse if possible
    try:
        parsed = json.loads(s)
        if isinstance(parsed, dict):
            msg = parsed.get("message")
            if isinstance(msg, dict):
                return msg.get("content", "").strip()
    except Exception:
        pass
    
    # Regex: message=Message(... content='...') or content='...' patterns
    m = re.search(r"message=Message\([^)]*content=(?P<q>['\"])(?P<content>.*?)(?P=q)", s, re.DOTALL)
    if m:
        return m.group("content").strip()
    
    m2 = re.search(r"content=(?P<q>['\"])(?P<content>.*?)(?P=q)", s, re.DOTALL)
    if m2:
        return m2.group("content").strip()
    
    # Fallback: return the original string
    return s.strip()


def execute_query(query: str, vector_store: VectorStore, metadata_store: MetadataStore, 
                 model_name: str = None, top_k: int = None, verbose: bool = True) -> dict:
    """
    End-to-end query execution: retrieve documents → generate answer → format with sources.
    
    Args:
        query: User's question
        vector_store: Loaded FAISS vector store
        metadata_store: Loaded metadata store
        model_name: LLM model to use (defaults to DEFAULT_OLLAMA_MODEL)
        top_k: Number of chunks to retrieve (defaults to DOCUMENT_CONFIG["top_k"])
        verbose: Print progress messages
    
    Returns:
        dict with keys: "query", "answer", "formatted_answer", "retrieved_chunks"
    """
    if model_name is None:
        model_name = DEFAULT_OLLAMA_MODEL
    if top_k is None:
        top_k = DOCUMENT_CONFIG["top_k"]
    
    if verbose:
        print(f"📖 Query: {query}")
    
    # Retrieve documents
    retrieved_chunks = retrieve_documents(query, vector_store, metadata_store, k=top_k)
    if verbose:
        print(f"✓ Retrieved {len(retrieved_chunks)} relevant chunks")
    
    # Generate answer
    if verbose:
        print(f"Generating answer with model: {model_name}")
    
    raw_response = generate_answer(query, retrieved_chunks, model_name=model_name, stream=False)
    answer_text = extract_content(raw_response)
    
    # Format with sources
    formatted_output = format_answer_with_sources(answer_text, retrieved_chunks)
    
    return {
        "query": query,
        "answer": answer_text,
        "formatted_answer": formatted_output,
        "retrieved_chunks": retrieved_chunks,
        "model": model_name
    }


def display_result(result: dict):
    """Pretty print query result with answer and sources."""
    print("\n" + "=" * 70)
    print("ANSWER WITH FORMATTED SOURCES:")
    print("=" * 70)
    print(result["formatted_answer"])
    print("=" * 70)

print("✓ Reusable helper functions loaded (extract_content, execute_query, display_result)")
print("✓ Configuration centralized in DOCUMENT_CONFIG, MODEL_CONFIG, and TOKEN_CONFIG")


✓ Reusable helper functions loaded (extract_content, execute_query, display_result)
✓ Configuration centralized in DOCUMENT_CONFIG, MODEL_CONFIG, and TOKEN_CONFIG


## Pipeline Integration and Testing

### Subtask:
Combine all components into a basic RAG pipeline and test with a sample document and query.

In [119]:
def check_if_embeddings_exist(save_dir: str) -> bool:
    """
    Check if embeddings and metadata already exist from a previous run.
    Returns True if all necessary files are present and valid.
    """
    import os
    
    required_files = [
        os.path.join(save_dir, "faiss_index.bin"),
        os.path.join(save_dir, "chunk_ids.pkl"),
        os.path.join(save_dir, "metadata.pkl")
    ]
    
    # Check if all files exist
    all_exist = all(os.path.exists(f) for f in required_files)
    
    if all_exist:
        try:
            # Try to load to verify integrity (silently)
            test_vs = VectorStore.load(save_dir, verbose=False)
            test_ms = MetadataStore.load(os.path.join(save_dir, "metadata.pkl"), verbose=False)
            
            # Check if they have content
            if test_vs.index.ntotal > 0 and len(test_ms.get_all_chunks()) > 0:
                return True
        except Exception as e:
            print(f"⚠️  Embeddings exist but failed to load: {e}")
    
    return False

def skip_embedding_if_exists(save_dir: str, sample_document_path: str):
    """
    Smart pipeline state manager:
    - If embeddings exist: Load them and skip document processing
    - If embeddings don't exist: Process document normally
    
    Returns: (should_skip_embedding, vector_store, metadata_store)
    """
    if check_if_embeddings_exist(save_dir):
        print("\n" + "=" * 70)
        print("⚡ FOUND PRECOMPUTED EMBEDDINGS - SKIPPING REDUNDANT API CALLS")
        print("=" * 70)
        print(f"✓ Embeddings already exist in {save_dir}")
        print(f"✓ Skipping: document loading, chunking, and embedding generation")
        print(f"✓ Directly loading from disk...\n")
        
        # Load silently since we're providing our own status messages
        vector_store = VectorStore.load(save_dir, verbose=False)
        metadata_store = MetadataStore.load(os.path.join(save_dir, "metadata.pkl"), verbose=False)
        
        print(f"✓ Loaded {vector_store.index.ntotal} precomputed embeddings")
        print(f"✓ Loaded {len(metadata_store.get_all_chunks())} document chunks")
        print(f"✓ Ready for querying!\n")
        
        return True, vector_store, metadata_store
    else:
        print(f"\n📄 No precomputed embeddings found. Processing document: {os.path.basename(sample_document_path)}")
        return False, None, None

print("✓ Smart embedding state checker loaded (avoids redundant API calls)")


✓ Smart embedding state checker loaded (avoids redundant API calls)


In [ ]:
# ===== INTELLIGENT PIPELINE EXECUTION (WITH SMART SKIP) =====

print("=" * 70)
print("RAG PIPELINE - Smart Embedding Reuse (Avoids Redundant API Calls)")
print("=" * 70)

# Unpack configuration
sample_document_path = DOCUMENT_CONFIG["sample_path"]
SAVE_DIR = DOCUMENT_CONFIG["save_dir"]
CHUNK_SIZE = DOCUMENT_CONFIG["chunk_size"]
CHUNK_OVERLAP = DOCUMENT_CONFIG["chunk_overlap"]

# Check if embeddings already exist from previous run
should_skip_embedding, vector_store, metadata_store = skip_embedding_if_exists(SAVE_DIR, sample_document_path)

if should_skip_embedding:
    # Embeddings exist - skip document processing and go straight to querying
    print("[1/2] ✓ SKIPPED - Using precomputed embeddings")
    print("[2/2] Testing retrieval & answer generation with formatted sources...")
else:
    # Embeddings don't exist - process document normally
    
    # Step 1: Load Document
    print("\n[1/7] Loading document...")
    try:
        document_data = load_document(sample_document_path)
        if isinstance(document_data, list):  # PDF returns list of (text, page_num)
            print(f"✓ Loaded PDF with {len(document_data)} pages")
            is_pdf = True
        elif isinstance(document_data, str):  # Text/DOCX returns string
            print(f"✓ Loaded document ({len(document_data)} characters)")
            is_pdf = False
            document_data = [(document_data, None)]  # Convert to same format
        else:
            raise ValueError("Failed to load document")
    except Exception as e:
        print(f"✗ Error loading document: {e}")
        raise

    # Step 2: Chunk Document
    print(f"\n[2/7] Chunking document (size={CHUNK_SIZE}, overlap={CHUNK_OVERLAP})...")
    if is_pdf:
        chunks_data = chunk_pdf_pages(document_data, chunk_size=CHUNK_SIZE, overlap=CHUNK_OVERLAP)
    else:
        chunks_data = chunk_document_semantic(document_data[0][0], chunk_size=CHUNK_SIZE, overlap=CHUNK_OVERLAP)

    print(f"✓ Created {len(chunks_data)} semantic chunks")

    # Step 3: Create Metadata Store
    print("\n[3/7] Building metadata store...")
    metadata_store = MetadataStore()
    document_chunks = []

    for chunk_text, page_num in chunks_data:
        chunk = DocumentChunk(
            text=chunk_text,
            source=os.path.basename(sample_document_path),
            page_number=page_num
        )
        metadata_store.add_chunk(chunk)
        document_chunks.append(chunk)

    metadata_store.index_chunks()

    # Step 4: Generate Embeddings (WITH OPTIMIZATIONS)
    print(f"\n[4/7] Generating embeddings using {EMBEDDING_METHOD}...")
    print("      ⚡ Enabling: caching, parallel processing, and batch optimization")
    generate_embeddings(document_chunks, batch_size=100, use_optimizations=True)

    # Step 5: Build Vector Store
    print("\n[5/7] Building vector store with cosine similarity...")
    vector_store = VectorStore(EMBEDDING_DIM)

    embeddings = np.array([chunk.embedding for chunk in document_chunks])
    chunk_ids = [chunk.chunk_id for chunk in document_chunks]

    vector_store.add_vectors(embeddings, chunk_ids)

    # Step 6: Save Everything to Disk
    print(f"\n[6/7] Saving to disk ({SAVE_DIR})...")
    os.makedirs(SAVE_DIR, exist_ok=True)
    vector_store.save(SAVE_DIR)
    metadata_store.save(os.path.join(SAVE_DIR, "metadata.pkl"))
    
    print(f"✓ Embeddings saved! Next run will reuse them automatically.")
    print(f"\n[7/7] Testing retrieval & answer generation with formatted sources...")

# ===== COMMON PATH (for both first run and subsequent runs) =====

query = "What are the Four Laws of Behavior Change?"
result = execute_query(query, vector_store, metadata_store, verbose=True)
display_result(result)

if should_skip_embedding:
    print(f"\n✓ Query complete! (Using cached embeddings)")
    print(f"  To regenerate embeddings: delete {SAVE_DIR}/ or set should_skip_embedding=False")
else:
    print(f"\n✓ Pipeline complete! Data saved to {SAVE_DIR}")
    print(f"  Embedding cache stored in: .embedding_cache/")
    print(f"  Next run will reuse embeddings automatically!")
    print(f"  To reload manually: vector_store = VectorStore.load('{SAVE_DIR}')")
    print(f"                      metadata_store = MetadataStore.load('{SAVE_DIR}/metadata.pkl')")


RAG PIPELINE - Smart Embedding Reuse (Avoids Redundant API Calls)

⚡ FOUND PRECOMPUTED EMBEDDINGS - SKIPPING REDUNDANT API CALLS
✓ Embeddings already exist in ./rag_data
✓ Skipping: document loading, chunking, and embedding generation
✓ Directly loading from disk...

✓ Loaded 660 precomputed embeddings
✓ Loaded 660 document chunks
✓ Ready for querying!

[1/2] ✓ SKIPPED - Using precomputed embeddings
[2/2] Testing retrieval & answer generation with formatted sources...

📖 Query: What are the Four Laws of Behavior Change?


✓ Retrieved 5 relevant chunks

Retrieved Sources:
  [1] Atomic_Habits_James_Clear.pdf (Page 52): The ultimate purpose of habits is to solve the problems of life with as little energy and effort as ...
  [2] Atomic_Habits_James_Clear.pdf (Page 155): Chapter Summary The 4th Law of Behavior Change is make it satisfying. We are more likely to repeat a...
  [3] Atomic_Habits_James_Clear.pdf (Page 51): transform these four steps into a practical framework that we can use to design good habits and elim...
  [4] Atomic_Habits_James_Clear.pdf (Page 150): for doing (or punished for doing) in the past. Positive emotions cultivate habits. Negative emotions...
  [5] Atomic_Habits_James_Clear.pdf (Page 51): Inversion of the 2nd law (Craving): Make it unattractive. Inversion of the 3rd law (Response): Make ...

Generating answer with model: gpt-oss:20b-cloud

ANSWER WITH FORMATTED SOURCES:
model='gpt-oss:20b' created_at='2025-10-25T21:19:56.239767959Z' done=True done_reason='stop' total_duration=1101

## Refinement and Evaluation

### Subtask:
Outline steps for refining and evaluating the RAG pipeline.

In [126]:
# ===== DEMO PREPARATION: UI-Ready Functions =====

# Global state for UI demos (initialized once)
_demo_state = {
    "vector_store": None,
    "metadata_store": None,
    "initialized": False,
}

def initialize_rag_demo():
    """
    Initialize RAG system for UI demo.
    Loads precomputed embeddings or processes document if needed.
    Safe to call multiple times - only initializes once.
    """
    if _demo_state["initialized"]:
        return _demo_state["vector_store"], _demo_state["metadata_store"]
    
    SAVE_DIR = DOCUMENT_CONFIG["save_dir"]
    sample_path = DOCUMENT_CONFIG["sample_path"]
    
    # Try to load precomputed embeddings
    if check_if_embeddings_exist(SAVE_DIR):
        print("✓ Loading precomputed embeddings...")
        _demo_state["vector_store"] = VectorStore.load(SAVE_DIR, verbose=False)
        _demo_state["metadata_store"] = MetadataStore.load(
            os.path.join(SAVE_DIR, "metadata.pkl"), verbose=False
        )
    else:
        print("Processing document (this may take a few minutes)...")
        # Run full pipeline
        should_skip, vs, ms = skip_embedding_if_exists(SAVE_DIR, sample_path)
        if not should_skip:
            # Full processing needed
            document_data = load_document(sample_path)
            is_pdf = isinstance(document_data, list)
            
            if is_pdf:
                chunks_data = chunk_pdf_pages(
                    document_data, 
                    chunk_size=DOCUMENT_CONFIG["chunk_size"],
                    overlap=DOCUMENT_CONFIG["chunk_overlap"]
                )
            else:
                chunks_data = chunk_document_semantic(
                    document_data, 
                    chunk_size=DOCUMENT_CONFIG["chunk_size"],
                    overlap=DOCUMENT_CONFIG["chunk_overlap"]
                )
            
            ms = MetadataStore()
            document_chunks = []
            for chunk_text, page_num in chunks_data:
                chunk = DocumentChunk(
                    text=chunk_text,
                    source=os.path.basename(sample_path),
                    page_number=page_num
                )
                ms.add_chunk(chunk)
                document_chunks.append(chunk)
            
            generate_embeddings(document_chunks, batch_size=100, use_optimizations=True)
            
            vs = VectorStore(EMBEDDING_DIM)
            embeddings = np.array([chunk.embedding for chunk in document_chunks])
            chunk_ids = [chunk.chunk_id for chunk in document_chunks]
            vs.add_vectors(embeddings, chunk_ids)
            
            os.makedirs(SAVE_DIR, exist_ok=True)
            vs.save(SAVE_DIR)
            ms.save(os.path.join(SAVE_DIR, "metadata.pkl"))
        else:
            vs = VectorStore.load(SAVE_DIR, verbose=False)
            ms = MetadataStore.load(os.path.join(SAVE_DIR, "metadata.pkl"), verbose=False)
        
        _demo_state["vector_store"] = vs
        _demo_state["metadata_store"] = ms
    
    _demo_state["initialized"] = True
    print("✓ RAG system ready for queries!")
    return _demo_state["vector_store"], _demo_state["metadata_store"]


def rag_query(user_query: str, model: str = None, top_k: int = None) -> str:
    """
    Simple interface for UI demos (Streamlit/Gradio).
    Takes a query string, returns formatted answer with sources.
    
    Args:
        user_query: User's question
        model: Optional model name (uses DEFAULT_OLLAMA_MODEL if None)
        top_k: Optional number of chunks to retrieve
    
    Returns:
        Formatted answer with source citations
    """
    vs, ms = _demo_state["vector_store"], _demo_state["metadata_store"]
    if vs is None or ms is None:
        raise ValueError("RAG system not initialized. Call initialize_rag_demo() first.")
    
    result = execute_query(user_query, vs, ms, model_name=model, top_k=top_k, verbose=False)
    return result["formatted_answer"]


def rag_query_with_details(user_query: str, model: str = None, top_k: int = None) -> dict:
    """
    Advanced interface returning full result details.
    Useful for apps that need answer + metadata.
    
    Args:
        user_query: User's question
        model: Optional model name
        top_k: Optional number of chunks to retrieve
    
    Returns:
        dict with: answer, formatted_answer, retrieved_chunks, model, query
    """
    vs, ms = _demo_state["vector_store"], _demo_state["metadata_store"]
    if vs is None or ms is None:
        raise ValueError("RAG system not initialized. Call initialize_rag_demo() first.")
    
    return execute_query(user_query, vs, ms, model_name=model, top_k=top_k, verbose=False)

print("✓ Demo prep functions loaded (initialize_rag_demo, rag_query, rag_query_with_details)")
print("Ready to create Streamlit/Gradio demo!")


✓ Demo prep functions loaded (initialize_rag_demo, rag_query, rag_query_with_details)
Ready to create Streamlit/Gradio demo!


In [123]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/63.5 MB ? eta -:--:--Downloading gradio-5.49.1-py3-none-any.whl (63.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 MB 4.3 MB/s  0:00:14m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 MB 4.3 MB/s  0:00:14
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.9 MB ? eta -:--:--Downloading annotated_doc-0.0.3-py3-none-any.whl (5.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 4.7 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 4.7 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/13.3 MB ? eta -:--:--Downloading python_multipart-0.0.20-py3-none-any.whl (24 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 4.8 MB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 4.8 MB/s  0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19/19 [gradio]18/19 [gradio]]lient]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19/19 

In [134]:
# ===== GRADIO DEMO INTERFACE =====

import gradio as gr
import socket

def is_port_available(port: int) -> bool:
    """Check if a port is available for use."""
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.bind(('', port))
            return True
    except OSError:
        return False

def find_available_port(start_port: int = 7860, max_attempts: int = 10) -> int:
    """Find an available port starting from start_port."""
    for offset in range(max_attempts):
        port = start_port + offset
        if is_port_available(port):
            return port
    raise RuntimeError(f"Could not find available port in range {start_port}-{start_port + max_attempts - 1}")

def rag_demo_interface(query: str, top_k: int = 5, model: str = None) -> str:
    """
    Gradio-compatible RAG interface.
    Takes user query and returns formatted answer with sources.
    """
    try:
        # Use default model if not specified
        if model is None or model.strip() == "":
            model = DEFAULT_OLLAMA_MODEL
        
        # Execute query with user-specified parameters
        result = rag_query_with_details(query, model=model, top_k=top_k)
        
        return result["formatted_answer"]
    
    except ValueError as e:
        # RAG system not initialized
        return f"❌ Error: {str(e)}\n\nPlease initialize the system first by running `initialize_rag_demo()`"
    except Exception as e:
        return f"❌ Error processing query: {str(e)}"


def create_gradio_demo():
    """
    Create and configure Gradio interface for RAG system.
    Returns the Gradio Blocks interface.
    """
    
    with gr.Blocks(
        title="📚 RAG Q&A Assistant",
        theme=gr.themes.Soft(),
        css="""
        .gr-box { border-radius: 12px; }
        .gr-button { border-radius: 8px; }
        .gr-textbox { border-radius: 8px; }
        """
    ) as demo:
        
        gr.Markdown("""
        # 📚 RAG Q&A Assistant
        
        Ask questions about your documents using semantic search + local LLM.
        **Powered by:** FAISS + Ollama + HuggingFace Embeddings
        """)
        
        with gr.Row():
            with gr.Column(scale=2):
                query_input = gr.Textbox(
                    label="❓ Your Question",
                    lines=4,
                    placeholder="e.g., What are the Four Laws of Behavior Change?",
                    interactive=True
                )
            
            with gr.Column(scale=1):
                top_k_slider = gr.Slider(
                    label="📊 Number of Sources",
                    minimum=1,
                    maximum=10,
                    value=5,
                    step=1,
                    interactive=True
                )
                
                model_dropdown = gr.Textbox(
                    label="🤖 Model (optional)",
                    value=DEFAULT_OLLAMA_MODEL,
                    placeholder="Leave blank for default",
                    interactive=True
                )
        
        # Submit button
        submit_btn = gr.Button(
            "🔍 Search",
            variant="primary",
            scale=1
        )
        
        # Output
        answer_output = gr.Markdown(
            label="📖 Answer with Sources",
            value="Your answer will appear here..."
        )
        
        # Connect button to function
        submit_btn.click(
            fn=rag_demo_interface,
            inputs=[query_input, top_k_slider, model_dropdown],
            outputs=answer_output
        )
        
        # Example questions
        gr.Examples(
            examples=[
                ["What are the Four Laws of Behavior Change?", 5, DEFAULT_OLLAMA_MODEL],
                ["How do habits compound over time?", 5, DEFAULT_OLLAMA_MODEL],
                ["What's the difference between systems and goals?", 5, DEFAULT_OLLAMA_MODEL],
                ["How can I break a bad habit?", 5, DEFAULT_OLLAMA_MODEL],
            ],
            inputs=[query_input, top_k_slider, model_dropdown],
            outputs=answer_output,
            fn=rag_demo_interface,
            cache_examples=False,
        )
        
        # Footer info
        gr.Markdown("""
        ---
        ### ℹ️ How It Works
        1. **Query Processing**: Your question is converted to embeddings
        2. **Semantic Search**: Top-K most relevant chunks are retrieved from the document
        3. **LLM Generation**: Local Ollama model generates answer based on retrieved chunks
        4. **Source Citation**: Answer includes [1], [2] citations linked to source documents
        
        ### 🚀 Performance
        - First run: ~5 minutes (document processing + embedding generation)
        - Subsequent runs: <5 seconds (embeddings cached)
        - Completely private & offline (when using local models)
        """)
    
    return demo


# Initialize RAG system and launch Gradio
print("🚀 Starting Gradio RAG Demo...")
print("=" * 70)

try:
    # Initialize RAG system
    print("📚 Initializing RAG system...")
    initialize_rag_demo()
    print("✓ RAG system ready!\n")
    
    # Find available port (handle port conflicts gracefully)
    preferred_port = 7860
    if not is_port_available(preferred_port):
        print(f"⚠️  Port {preferred_port} is already in use. Finding available port...")
        available_port = find_available_port(preferred_port)
        print(f"✓ Using port {available_port} instead\n")
    else:
        available_port = preferred_port
        print(f"✓ Port {preferred_port} is available\n")
    
    # Create and launch Gradio interface
    gradio_demo = create_gradio_demo()
    
    print("🌐 Launching Gradio interface...")
    print("=" * 70)
    print(f"📖 Access the demo at: http://127.0.0.1:{available_port}")
    print("🌍 Public link (share=True): will be printed below")
    print("=" * 70)
    
    gradio_demo.launch(
        share=True,                    # Generate public link
        server_name="0.0.0.0",
        server_port=available_port,   # Use dynamically selected port
    )
    
except Exception as e:
    print(f"\n❌ Error launching Gradio demo: {e}")
    print("\nTroubleshooting:")
    print("  1. If port conflict: a new port will be automatically selected")
    print("  2. Make sure RAG system initialization succeeded")
    print("  3. Check if Gradio is still running from a previous session")
    import traceback
    traceback.print_exc()


🚀 Starting Gradio RAG Demo...
📚 Initializing RAG system...
✓ RAG system ready!

⚠️  Port 7860 is already in use. Finding available port...
✓ Using port 7861 instead

🌐 Launching Gradio interface...
📖 Access the demo at: http://127.0.0.1:7861
🌍 Public link (share=True): will be printed below
🌐 Launching Gradio interface...
📖 Access the demo at: http://127.0.0.1:7861
🌍 Public link (share=True): will be printed below
* Running on local URL:  http://0.0.0.0:7861
* Running on local URL:  http://0.0.0.0:7861
* Running on public URL: https://0ad972a84275c83c70.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
* Running on public URL: https://0ad972a84275c83c70.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy t

[LLM Config] Sources: 8 | Max tokens: 2048


In [131]:

# ===== TEST NEW TOKEN LIMITS =====
# Test the dynamic token limits with different numbers of sources

print("=" * 70)
print("🧪 TESTING DYNAMIC TOKEN LIMITS FOR LLM RESPONSES")
print("=" * 70)

# Test with different numbers of sources
test_queries = [
    ("What are the Four Laws of Behavior Change?", 3, "3 sources"),
    ("How do habits compound over time?", 5, "5 sources"),
    ("Explain the habit loop and its components", 8, "8 sources"),
    ("What are the best strategies for building good habits?", 10, "10 sources (max)"),
]

print("\n📊 Token Limit Configuration:")
print(f"   Base tokens: {TOKEN_CONFIG['base']} (increased from 256)")
print(f"   Per source: {TOKEN_CONFIG['per_source']} tokens/source")
print(f"   Max tokens: {TOKEN_CONFIG['max']} (absolute cap)")
print(f"\n📐 Formula: num_predict = min(base + num_sources × per_source, max)")

print("\n" + "=" * 70)
print("Expected token limits per test case:")
print("=" * 70)

for query, num_sources, description in test_queries:
    expected_tokens = min(
        TOKEN_CONFIG['base'] + (num_sources * TOKEN_CONFIG['per_source']),
        TOKEN_CONFIG['max']
    )
    print(f"   {description:20} → {expected_tokens:4} max tokens")

print("\n✅ IMPROVEMENTS MADE:")
print("   ✓ Token limit increased from 256 → 512 base")
print("   ✓ Dynamic scaling: +200 tokens per additional source")
print("   ✓ 3 sources  → 912 tokens   (3.6x increase)")
print("   ✓ 5 sources  → 1512 tokens  (5.9x increase)")
print("   ✓ 10 sources → 2048 tokens  (max)")
print("\n💡 TRY NOW:")
print("   1. In Gradio, set 'Number of Sources' to 10")
print("   2. Ask a question with the default model")
print("   3. Answers will now be MUCH fuller with proper citations!")
print("   4. Watch for '[LLM Config] Sources: 10 | Max tokens: 2048' in output")
print("\n" + "=" * 70)


🧪 TESTING DYNAMIC TOKEN LIMITS FOR LLM RESPONSES

📊 Token Limit Configuration:
   Base tokens: 512 (increased from 256)
   Per source: 200 tokens/source
   Max tokens: 2048 (absolute cap)

📐 Formula: num_predict = min(base + num_sources × per_source, max)

Expected token limits per test case:
   3 sources            → 1112 max tokens
   5 sources            → 1512 max tokens
   8 sources            → 2048 max tokens
   10 sources (max)     → 2048 max tokens

✅ IMPROVEMENTS MADE:
   ✓ Token limit increased from 256 → 512 base
   ✓ Dynamic scaling: +200 tokens per additional source
   ✓ 3 sources  → 912 tokens   (3.6x increase)
   ✓ 5 sources  → 1512 tokens  (5.9x increase)
   ✓ 10 sources → 2048 tokens  (max)

💡 TRY NOW:
   1. In Gradio, set 'Number of Sources' to 10
   2. Ask a question with the default model
   3. Answers will now be MUCH fuller with proper citations!
   4. Watch for '[LLM Config] Sources: 10 | Max tokens: 2048' in output



In [132]:

# ===== MANUAL TEST: Verify Token Limit Fix =====

print("\n" + "=" * 70)
print("✅ QUICK TEST: Manual Query with 10 Sources")
print("=" * 70)

# Test directly with the loaded RAG system (no Gradio needed)
if _demo_state.get("initialized"):
    test_query = "What are the four laws of behavior change?"
    print(f"\n📝 Query: {test_query}")
    print(f"📊 Testing with: 10 sources (should show '[LLM Config] Sources: 10 | Max tokens: 2048')\n")
    
    result = rag_query_with_details(test_query, top_k=10)
    
    print(f"\n✅ Answer length: {len(result['answer'])} characters")
    print(f"✅ Sources retrieved: {len(result['retrieved_chunks'])}")
    print("\n" + "=" * 70)
    print("FORMATTED ANSWER WITH ALL SOURCES:")
    print("=" * 70)
    print(result["formatted_answer"][:1500])  # Show first 1500 chars
    print("\n... (truncated for display)")
else:
    print("⚠️  RAG system not initialized. Run the pipeline and demo prep cells first.")



✅ QUICK TEST: Manual Query with 10 Sources

📝 Query: What are the four laws of behavior change?
📊 Testing with: 10 sources (should show '[LLM Config] Sources: 10 | Max tokens: 2048')

[LLM Config] Sources: 10 | Max tokens: 2048

✅ Answer length: 592 characters
✅ Sources retrieved: 10

FORMATTED ANSWER WITH ALL SOURCES:
The Four Laws of Behavior Change are a simple framework for building good habits and breaking bad ones. They are:  \n\n1. **Make it obvious** – design cues that clearly signal the desired behavior.  \n2. **Make it attractive** – increase the appeal or craving for the habit.  \n3. **Make it easy** – reduce friction so the response can happen with minimal effort.  \n4. **Make it satisfying** – provide immediate reward so the behavior is repeated.  \n\nThese laws map onto the cue‑craving‑response‑reward habit loop and are intended to make habit formation effortless and sustainable. [1][3]

📚 SOURCES:

[1] Atomic_Habits_James_Clear.pdf (Page 52):
    "The ultimate purpose o

## Loading Previously Saved Data

If you've already processed a document and saved the vector store, you can load it quickly without re-processing:

In [ ]:
# Load previously saved vector store and metadata
try:
    SAVE_DIR = DOCUMENT_CONFIG["save_dir"]
    vector_store = VectorStore.load(SAVE_DIR)
    metadata_store = MetadataStore.load(os.path.join(SAVE_DIR, "metadata.pkl"))
    
    # Test with a new query using reusable helper
    query = "How do habits compound over time?"
    result = execute_query(query, vector_store, metadata_store, verbose=True)
    display_result(result)
    
except FileNotFoundError:
    print(f"No saved data found in {DOCUMENT_CONFIG['save_dir']}. Run the pipeline cell first!")


Vector store loaded from ./rag_data (660 vectors)
Metadata store loaded from ./rag_data/metadata.pkl (660 chunks)
📖 Query: How do habits compound over time?

✓ Retrieved 5 relevant chunks (using cached embeddings)

Generating answer with model: gpt-oss:20b-cloud
✓ Retrieved 5 relevant chunks (using cached embeddings)

Generating answer with model: gpt-oss:20b-cloud



FORMATTED ANSWER WITH SOURCES:
Habits work like compound interest: a small daily improvement—say 1\u202f% better each day—multiplies over time, producing a result that can be dozens of times larger after a year (≈37×) [1]. The key is the rate and frequency of the behavior; repeated successful attempts build the habit until it crosses the “habit line” and becomes automatic [2]. By tracking tiny gains or losses day‑to‑day, you can predict where you’ll be years later, because those small choices accumulate and magnify over time, making good habits an ally of time while bad habits become its enemy [3].

📚 SOURCES:

[1] Atomic_Habits_James_Clear.pdf (Page 20):
    "FIGURE 1: The effects of small habits compound over time. For example, if you can get just 1 percent better each day, you’ll end up with results that are nearly 37 times better after one year. Habits ..."

[2] Atomic_Habits_James_Clear.pdf (Page 120):
    "What matters is the rate at which you perform the behavior. You could do 

This step involves:
- **Improving Chunking:** Experiment with different chunking strategies (e.g., semantic chunking, recursive text splitting) and chunk sizes.
- **Optimizing Retrieval:** Explore different vector store implementations (e.g., specialized databases like Pinecone, Weaviate) and retrieval algorithms. Fine-tune the `k` parameter for retrieval.
- **Enhancing Prompting:** Develop more sophisticated prompting techniques for the language model to improve answer quality and incorporate citations.
- **Evaluation Metrics:** Define metrics to evaluate the performance of the RAG pipeline, such as:
    - **Relevance:** How relevant are the retrieved chunks to the query?
    - **Faithfulness:** How well does the generated answer align with the information in the retrieved chunks?
    - **Answer Quality:** Is the generated answer coherent, accurate, and helpful?
- **Testing with Diverse Data:** Test the pipeline with a variety of document types and query styles.

## Finish task